In [44]:
import re
import nltk
from nltk.translate import AlignedSent, IBMModel1

import pandas as pd
from transformers import MBartForConditionalGeneration, MBartTokenizer
from torch.utils.data import Dataset, DataLoader
import torch
from transformers import AdamW
from sklearn.model_selection import train_test_split

In [45]:
import pandas as pd

df = pd.read_parquet("hf://datasets/SKNahin/bengali-transliteration-data/data/train-00000-of-00001.parquet")

In [46]:
df.head(3)

,bn,rm
0,স্ক্রোল করে ২০/৩০ সেকেন্ড এর ভিডিও পান নাই???,scroll kore 20/30 second er video pann nai???
1,ও গুলা টরেন্ট সাইট এ পাবেন,o gula Torrent site e paben
2,ভক্কর চক্কর পোস্ট একটা করলেই এপ্রুভড.… নিশ্চই ...,vokkor chokkor post akta korlei approved…. nis...


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5006 entries, 0 to 5005
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   bn      5006 non-null   object
 1   rm      5006 non-null   object
dtypes: object(2)
memory usage: 78.3+ KB


In [47]:
df['rm_count'] = df['rm'].apply(len)
df['bn_count'] = df['bn'].apply(len)

In [40]:
df.head(5)

,bn,rm,rm_count,bn_count
0,স্ক্রোল করে ২০/৩০ সেকেন্ড এর ভিডিও পান নাই???,scroll kore 20/30 second er video pann nai???,45,45
1,ও গুলা টরেন্ট সাইট এ পাবেন,o gula Torrent site e paben,27,26
2,ভক্কর চক্কর পোস্ট একটা করলেই এপ্রুভড.… নিশ্চই ...,vokkor chokkor post akta korlei approved…. nis...,63,56
3,আমি টেস্ট করেই কোড দিছি…,ami test koreii code disi…,26,24
4,"এতো কষ্টের কি আছে সাকিবওয়াপ.টক,সাকিবওয়াপ.মল&এআ...","eto koster ki ache shakibwap.tk,shakibwap.ml&a...",90,86


In [7]:
df[df['rm_count'] <= 2]

,bn,rm,rm_count,bn_count
1372,হা,ha,2,2
2637,হুম,Hm,2,3
2760,হ,ho,2,1


In [8]:
df.describe()

,rm_count,bn_count
count,5006.000000,5006.000000
mean,36.082901,35.398522
std,25.347631,24.663813
min,2.000000,1.000000
25%,20.000000,20.000000
50%,30.000000,29.000000
75%,44.000000,43.000000
max,253.000000,246.000000


For cleaning the data in the dataset we removed the sentences that are long than 128 character. There is 64 entries in the data which is larger than 128 character which makes up 1.27% of the data set.

As the outliers are less than <5% they can be safely removed without affecting the total dataset.

In [48]:
# df[df['rm_count'] > 128]

df=df[df['rm_count'] < 128]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4942 entries, 0 to 5005
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   bn        4942 non-null   object
 1   rm        4942 non-null   object
 2   rm_count  4942 non-null   int64 
 3   bn_count  4942 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 193.0+ KB


In [14]:
final_df = df[['bn', 'rm']]

In [16]:
!pip install transformers datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


Spliting the data in 80/20 for train and test dataset.

In [49]:
df = final_df.copy()

df = df.rename(columns={'rm': 'source_text', 'bn': 'target_text'})

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['source_text'].tolist(),
    df['target_text'].tolist(),
    test_size=0.2,
    random_state=42
)

defining the pre trained model mbart50 due to it's wide uses in translation of language.

In [28]:
model_name = "facebook/mbart-large-50"
tokenizer = MBartTokenizer.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

source_lang = "en_XX"
target_lang = "bn_BD"


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'MBart50Tokenizer'. 
The class this function is called from is 'MBartTokenizer'.


Define the class to tokenize the data set.

In [ ]:
class TranslationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, source_lang, target_lang, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.source_lang = source_lang
        self.target_lang = target_lang
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        source = self.texts[idx]
        target = self.labels[idx]

        source_encodings = self.tokenizer(
            source, return_tensors="pt", truncation=True, padding="max_length", max_length=self.max_len
        )
        target_encodings = self.tokenizer(
            target, return_tensors="pt", truncation=True, padding="max_length", max_length=self.max_len
        )

        return {
            "input_ids": source_encodings["input_ids"].squeeze(),
            "attention_mask": source_encodings["attention_mask"].squeeze(),
            "labels": target_encodings["input_ids"].squeeze(),
        }



# Create PyTorch Datasets
train_dataset = TranslationDataset(train_texts, train_labels, tokenizer, source_lang, target_lang)
val_dataset = TranslationDataset(val_texts, val_labels, tokenizer, source_lang, target_lang)

In [30]:
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)


In [31]:

# Step 6: Training Loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=3e-5)

epochs = 5
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        optimizer.zero_grad()

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader)}")

# Save the Model
model.save_pretrained("banglish-to-bengali-mbart")
tokenizer.save_pretrained("banglish-to-bengali-mbart")

print("Model saved!")

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/5, Loss: 1.5580093028719078
Epoch 2/5, Loss: 0.24708909751799055
Epoch 3/5, Loss: 0.12733600954125504
Epoch 4/5, Loss: 0.10666702024475067
Epoch 5/5, Loss: 0.05990847916957623


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


Model saved!


In [32]:
model.eval()
with torch.no_grad():
    for batch in val_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask)
        for idx, output in enumerate(outputs):
            print("Input:", tokenizer.decode(input_ids[idx], skip_special_tokens=True))
            print("Prediction:", tokenizer.decode(output, skip_special_tokens=True))
            print("Target:", tokenizer.decode(labels[idx], skip_special_tokens=True))

Input: hoi na keno
Prediction: এই না কেনো
Target: হয় না কেন
Input: 15k budget a 635 moteo jay na
Prediction: ৪০ক বাজেট এ ৬৩৫ মোটেও যায় না
Target: ১৫ক বাজেট এ ৬৩৫ মোটেও যায় না
Input: Sorry vai dite vule gesi
Prediction: হুম ভাই দিতে ভুলে গেছি
Target: সরি ভাই দিতে ভুলে গেছি
Input: fast kaj kore ki?
Prediction: এই কাজ করে কি?
Target: ফাস্ট কাজ করে কি?
Input: Apnar prime account ta ektu diben
Prediction: আপনের প্রাইম একাউন্ট টা একটু দিবেন
Target: আপনার প্রাইম একাউন্ট টা একটু দিবেন
Input: windows defender off kore nen
Prediction: এইচএসডি সার্ভার অফ করে নেন
Target: উইন্ডোজ ডিফেন্ডার অফ করে নেন
Input: Doctor na hole agulo janen kivabe
Prediction: নোট না হলে এত ঝামেলার কীভাবে
Target: ডক্টর না হলে এইগুলো জানেন কিভাবে
Input: telegram group e join koren ..paben oikhane amake..
Prediction: টেলিগ্রাম গ্রুপ এ জয়েন করেন ..পাবেন ওখানে আসেন..
Target: টেলিগ্রাম গ্রুপ এ জয়েন করেন ..পাবেন ঐখানে আমাকে..
Input: 7.1 te support kore
Prediction: ৭৭.১ তে সাপোর্ট করে
Target: ৭.১ তে সাপোর্ট করে
Input: Apnar 

KeyboardInterrupt: 

In [33]:
import torch
from transformers import MBartForConditionalGeneration, MBartTokenizer

# Step 1: Load the Saved Model and Tokenizer
model_path = "banglish-to-bengali-mbart"  # Path to your saved model
model = MBartForConditionalGeneration.from_pretrained(model_path)
tokenizer = MBartTokenizer.from_pretrained(model_path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# Step 2: Define a Function for Translation
def translate_banglish_to_bengali(text, model, tokenizer, max_length=128):
    # Tokenize the input text
    input_encodings = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=max_length
    ).to(device)

    # Generate the output
    outputs = model.generate(
        input_ids=input_encodings["input_ids"],
        attention_mask=input_encodings["attention_mask"],
        max_length=max_length,
        num_beams=4,  # Beam search for better results
        early_stopping=True
    )


    translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return translated_text

test_sentences = [
    "ami jabo school-e",
    "tumi kemon acho?",
    "imran ekta faul."
]

for sentence in test_sentences:
    bengali_translation = translate_banglish_to_bengali(sentence, model, tokenizer)
    print(f"Banglish: {sentence}")
    print(f"Bengali: {bengali_translation}")
    print("-" * 50)


Banglish: ami jabo school-e
Bengali: আমি যাব স্কুল-এ
--------------------------------------------------
Banglish: tumi kemon acho?
Bengali: ভাই কেমন আছি?
--------------------------------------------------
Banglish: imran ekta faul.
Bengali: এইটার একটা ফাউল।
--------------------------------------------------
